# Homework 2 
**Wed, Sep 28 2022**

*Jackson Rudoff* 


In [108]:
import pandas as pd
import numpy as np

In [109]:
expeditions = pd.read_csv('./data/expeditions.csv')
members = pd.read_csv('./data/members.csv')
peaks = pd.read_csv('./data/peaks.csv')

### Question 1

#### A) Get proportion of unclimbed peaks

Rather simple approach here, get counts of unclimbed and divide by length of column.

In [110]:
# Get proportion of peaks unclimbed
unclimbed_one = (peaks[peaks['climbing_status'] == 'Unclimbed'].value_counts('climbing_status')/len(peaks))
unclimbed_one # around 27% are still unclimbed


climbing_status
Unclimbed    0.271368
dtype: float64

In [312]:
#OR in Numpy (as discussed in recitation)

np.where(peaks['climbing_status'] == 'Unclimbed', 1,0).mean()

0.27136752136752135

Either way, ~27% are unclimbed. 

#### B) Average Height of Climbed & Unclimbed Peaks

Also straightforward, simply groupby() climbing status and then call the **.mean()** method.

In [112]:
avg_height = peaks.groupby('climbing_status')['height_metres'].mean()
avg_height # 6706.3 for Climbed, 6523.3 for Unclimbed

climbing_status
Climbed      6706.284457
Unclimbed    6523.330709
Name: height_metres, dtype: float64

### Question 2

#### A) Find Percentage of Sherpas Who Were Hired

This one begins by filtering for those with Nepalese citizenship, which I did through bracket subsetting:

In [113]:
members_citizenship = members[members['citizenship'] == 'Nepal'] # get Sherpas
members_citizenship

,expedition_id,member_id,peak_id,peak_name,year,season,sex,age,citizenship,expedition_role,...,highpoint_metres,success,solo,oxygen_used,died,death_cause,death_height_metres,injured,injury_type,injury_height_metres
24,AMAD79101,AMAD79101-18,AMAD,Ama Dablam,1979,Spring,M,23.0,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
48,AMAD79301,AMAD79301-26,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
49,AMAD79301,AMAD79301-24,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
50,AMAD79301,AMAD79301-25,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
82,AMAD81101,AMAD81101-11,AMAD,Ama Dablam,1981,Spring,M,25.0,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76506,PUMO19102,PUMO19102-01,PUMO,Pumori,2019,Spring,M,35.0,Nepal,Leader,...,7138.0,True,False,False,False,NaN,NaN,False,NaN,NaN
76507,PUMO19102,PUMO19102-02,PUMO,Pumori,2019,Spring,M,52.0,Nepal,H-A Worker,...,7138.0,True,False,False,False,NaN,NaN,False,NaN,NaN
76513,PUTH19101,PUTH19101-06,PUTH,Putha Hiunchuli,2019,Spring,M,NaN,Nepal,H-A Worker,...,6350.0,False,False,False,False,NaN,NaN,False,NaN,NaN
76514,PUTH19101,PUTH19101-07,PUTH,Putha Hiunchuli,2019,Spring,M,44.0,Nepal,H-A Worker,...,6350.0,False,False,False,False,NaN,NaN,False,NaN,NaN


Then get **value_counts()** for hired/unhired and divide by length of column. 

In [316]:
members_cit_hired = members_citizenship['hired'].value_counts()[True]/len(members_citizenship)
members_cit_hired # hired percentage ~ 93.4%

0.9340563991323211

#### B) Min, Max, Mean Age of Sherpas

This one is a little mroe complex. Subset by age/citizenship, then groupby() citizenship to set up the crosstab. Then use agg to wrap the desired summary statistics: 

In [115]:
members_cit_age = members_citizenship[['age','citizenship']].groupby('citizenship').agg(['min', 'max', 'mean'])
members_cit_age

age                 
              min   max       mean
citizenship                       
Nepal        12.0  85.0  32.885062

### Question 3


#### A) Find First Record of Woman Summiting a Peak

First step is to merge the data, which can be done with a left-join on the provided id variables. 

In [296]:
merged_data = pd.merge(pd.merge(peaks, expeditions, 
         left_on = 'peak_id', right_on = 'peak_id'), members[members['hired'] == False],
         left_on = 'expedition_id', right_on = 'expedition_id', indicator = True)


merged_data.duplicated() # check for duplicated rows

0        False
1        False
2        False
3        False
4        False
         ...  
60643    False
60644    False
60645    False
60646    False
60647    False
Length: 60648, dtype: bool

I like to eliminate some of the duplicated columns, and I found this method a while back for doing so that uses Regex to remove columns ending in a specific string.

In [297]:
merged_data.drop(list(merged_data.filter(regex = '_x')), axis = 1, inplace = True) #method I found for removing '_x' columns
merged_data

,peak_alternative_name,height_metres,climbing_status,first_ascent_year,first_ascent_country,first_ascent_expedition_id,expedition_id,peak_name_y,basecamp_date,highpoint_date,...,success,solo,oxygen_used_y,died,death_cause,death_height_metres,injured,injury_type,injury_height_metres,_merge
0,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88101,Ama Dablam,1988-03-22,1988-04-10,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
1,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88101,Ama Dablam,1988-03-22,1988-04-10,...,False,False,False,False,NaN,NaN,True,AMS,NaN,both
2,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88101,Ama Dablam,1988-03-22,1988-04-10,...,False,False,False,False,NaN,NaN,True,Falling rock / ice,NaN,both
3,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88101,Ama Dablam,1988-03-22,1988-04-10,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
4,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88102,Ama Dablam,1988-03-23,1988-04-15,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60643,Khumbhakarna East,7460,Unclimbed,NaN,NaN,NaN,JANE18301,Jannu East,2018-09-19,2018-09-19,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
60644,Khumbhakarna East,7460,Unclimbed,NaN,NaN,NaN,JANE18301,Jannu East,2018-09-19,2018-09-19,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
60645,Khumbhakarna East,7460,Unclimbed,NaN,NaN,NaN,JANE19101,Jannu East,2019-03-11,2019-03-28,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
60646,Khumbhakarna East,7460,Unclimbed,NaN,NaN,NaN,JANE19101,Jannu East,2019-03-11,2019-03-28,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both


The set up crosstab that shows the name of the peak, its height, and the year it was summited. 

In [298]:
first_female = merged_data[(
    merged_data['sex'] == 'F') & (merged_data['success'] == True)].nsmallest(
    1, 'year_y', keep = 'all')[['peak_name', 'height_metres', 'year_y']]

first_female

,peak_name,height_metres,year_y
57666,Tengkoma,6215,1949
58094,Dzanye,6581,1949


#### B) Cross-Tab of Success, Oxygen Use, and Death

Using the merged dataframe, this can be done in one line as well through a **lambda**. Since you're working with a boolean variable, you can just call mean() to get the desired proportions. 

In [299]:
prop_tab = (merged_data[['success', 'oxygen_used_y', 'died', 'sex']]
.groupby('sex')
.apply(lambda x: x.mean(numeric_only=False))
)

prop_tab

,success,oxygen_used_y,died
sex,,,
F,0.327424,0.216328,0.006720
M,0.313204,0.175203,0.013718


### Question 4

#### A) Count Number of Expeditions, Members, and Death Rates for Hired/Unhired

Before getting to work on this question, a fellow student noticed a relevant data error related to members/hired-staff. There are rows in the data where the # of deaths is greater than the # of people recorded on the expedition. Step one, then, is to remove these. 

In [194]:
#Drop rows based on condition
expeditions.drop(expeditions[expeditions['member_deaths'] > expeditions['members']].index, inplace=True)
expeditions.drop(expeditions[expeditions['hired_staff_deaths'] > expeditions['hired_staff']].index, inplace=True)

Next, I want to convert the years in the data to decades, select my columns of interest, and then group by decade.

In [195]:
expeditions['decade'] = ((expeditions['year']//10)*10)
expeditions_grouped = (expeditions[['decade','expedition_id', 'members', 'member_deaths', 'hired_staff', 'hired_staff_deaths']]
                       .drop_duplicates(subset = ['expedition_id']) #found a duplicate here, want to eliminate
                       .groupby('decade'))

There's a couple ways to approach aggregating these summary statistics, but I'm going to break it up a bit. First, get total expeditions and members.

In [196]:
exped_totals = expeditions_grouped[['expedition_id', 'members']].agg({
   'expedition_id' : 'count', 
   'members' : 'sum'}).rename(
      columns = {'expedition_id' : 'Expeditions', 'members' : 'Members'})
      
exped_totals

,Expeditions,Members
decade,,
1900,4,9
1910,3,3
1920,10,51
1930,34,152
1940,15,53
1950,168,856
1960,160,973
1970,357,3465
1980,1184,9993


Then, get death rates by decade by summing the death totals and dividing by sum of members/hired staff. 

In [300]:
death_rate1 = (100*(expeditions_grouped['member_deaths'].sum())/(expeditions_grouped['members'].sum()))
death_rate2 = (100*(expeditions_grouped['hired_staff_deaths'].sum())/(expeditions_grouped['hired_staff'].sum()))
death_rates = pd.concat([death_rate1, death_rate2], axis= 1, keys = ['Member_Death_Rates', 'Hired_Death_Rates'])

Kind of a roundabout procedure, but it will play nicely with a merge now. 

In [301]:
summarized_exped = pd.merge(exped_totals, death_rates, on = 'decade')

summarized_exped

,Expeditions,Members,Member_Death_Rates,Hired_Death_Rates
decade,,,,
1900,4,9,11.111111,100.000000
1910,3,3,0.000000,NaN
1920,10,51,7.843137,8.403361
1930,34,152,1.315789,1.465201
1940,15,53,0.000000,0.000000
1950,168,856,1.869159,1.196809
1960,160,973,2.261048,0.932642
1970,357,3465,2.683983,2.336235
1980,1184,9993,1.991394,1.407529


#### Key Findings:

1. Expeditions have increased *a lot* in the last 4 decades. This is a phenomenon I've read about, but seeing it laid out this way is incredible. I am curious as to what the upward bound of the number of expeditions that could be done per-year is. 
2. Anyone hired for an expedition in the 1900s died, which is pretty absurd, but also likely a sign of how expedition leaders treated or worked with their guides. I am guessing that at this point, these expeditions were largely exploratory and the people undertaking them did not have great knowledge of the peaks they were climbing. 
3. The death rates spiked in the 60s and 70s, along with the number of expeditions. Since then, the death rates appear to have gone down each decade, despite more people attempting expeditions. I would hazard a guess that routes became more optimized from the 80s onward, thanks to discoveries and successful attempts by other climbers. It is also possible that better regulations were put in place to ensure save practices by those who attempted climbs.

#### B) Length of Expeditions vs Death Rates

We're working with date objects here which will make things a bit complicated. I decided to convert the two columns with start/end dates to datetime objects. 

In [240]:
expeditions['length_of_expedition'] = pd.to_datetime(expeditions['termination_date']) - pd.to_datetime(expeditions['basecamp_date'])
dates = expeditions[
    ['peak_id', 'expedition_id','basecamp_date', 'termination_date', 'length_of_expedition']
    ].dropna() #select only relevant columns & remove NA rows

dates


,peak_id,expedition_id,basecamp_date,termination_date,length_of_expedition
1,ANN2,ANN269301,1969-09-25,1969-10-26,31 days
3,ANN2,ANN278301,1978-09-08,1978-10-05,27 days
5,ANN2,ANN280101,1980-03-25,1980-05-01,37 days
8,ANN2,ANN281301,1981-09-04,1981-10-17,43 days
9,ANN2,ANN282301,1982-09-14,1982-10-08,24 days
...,...,...,...,...,...
10359,PUMO,PUMO19101,2019-04-04,2019-05-27,53 days
10360,PUMO,PUMO19102,2019-04-14,2019-05-25,41 days
10361,PUTH,PUTH19101,2019-05-09,2019-05-17,8 days
10362,RATC,RATC19101,2019-04-11,2019-05-03,22 days


We have our dates and lengths, but we still have to fix up the data a bit to get our z-scores.

In [241]:
dates['length_of_expedition'] = dates['length_of_expedition'].dt.days #make integer to be safe

In [242]:
dates

,peak_id,expedition_id,basecamp_date,termination_date,length_of_expedition
1,ANN2,ANN269301,1969-09-25,1969-10-26,31
3,ANN2,ANN278301,1978-09-08,1978-10-05,27
5,ANN2,ANN280101,1980-03-25,1980-05-01,37
8,ANN2,ANN281301,1981-09-04,1981-10-17,43
9,ANN2,ANN282301,1982-09-14,1982-10-08,24
...,...,...,...,...,...
10359,PUMO,PUMO19101,2019-04-04,2019-05-27,53
10360,PUMO,PUMO19102,2019-04-14,2019-05-25,41
10361,PUTH,PUTH19101,2019-05-09,2019-05-17,8
10362,RATC,RATC19101,2019-04-11,2019-05-03,22


Now we can **groupby()** peak_id and wrap the zscore calculation as a **lambda** function.

In [243]:
dates['peak_length_score'] = dates[['peak_id', 'length_of_expedition']].groupby(
                                                        'peak_id'
                                                             ).transform(lambda x: (x - x.mean()) / x.std())

dates[['peak_id', 'expedition_id','peak_length_score']]


,peak_id,expedition_id,peak_length_score
1,ANN2,ANN269301,-0.074159
3,ANN2,ANN278301,-0.400456
5,ANN2,ANN280101,0.415288
8,ANN2,ANN281301,0.904734
9,ANN2,ANN282301,-0.645179
...,...,...,...
10359,PUMO,PUMO19101,4.141132
10360,PUMO,PUMO19102,2.813516
10361,PUTH,PUTH19101,-0.925485
10362,RATC,RATC19101,0.653566


We also want to know if longer expeditions are associated with more deaths. There's a few ways to think about this (without statistical tests), and the most basic is to just look at the rates, sorted with length of expedition. 

In [224]:
deaths_vs_length= expeditions[
    ['peak_id', 'expedition_id', 'length_of_expedition', 'member_deaths', 'hired_staff_deaths']].dropna()

deaths_vs_length

,peak_id,expedition_id,length_of_expedition,member_deaths,hired_staff_deaths
1,ANN2,ANN269301,31 days,0,0
3,ANN2,ANN278301,27 days,0,0
5,ANN2,ANN280101,37 days,1,0
8,ANN2,ANN281301,43 days,1,0
9,ANN2,ANN282301,24 days,0,0
...,...,...,...,...,...
10359,PUMO,PUMO19101,53 days,0,0
10360,PUMO,PUMO19102,41 days,0,0
10361,PUTH,PUTH19101,8 days,0,0
10362,RATC,RATC19101,22 days,0,0


Now lets see if expedition length is associated with deaths in anyway. Doing a basic sort should paint the picture:

In [230]:
deaths_vs_length.sort_values('length_of_expedition')

,peak_id,expedition_id,length_of_expedition,member_deaths,hired_staff_deaths
7660,ANN4,ANN412302,0 days,0,0
5469,MANA,MANA06306,0 days,0,0
4729,CHOY,CHOY04331,0 days,0,0
6873,CHOY,CHOY10304,0 days,0,0
3027,PARC,PARC60301,0 days,0,0
...,...,...,...,...,...
232,ANN1,ANN185401,104 days,0,0
7495,EVER,EVER12128,107 days,0,0
8075,AMAD,AMAD13110,132 days,0,0
468,EVER,EVER87401,133 days,0,0


In [232]:
deaths_vs_length.sort_values('member_deaths')

,peak_id,expedition_id,length_of_expedition,member_deaths,hired_staff_deaths
1,ANN2,ANN269301,31 days,0,0
6856,MANA,MANA10109,24 days,0,0
6855,MANA,MANA10107,29 days,0,0
6854,MANA,MANA10106,36 days,0,0
6853,MANA,MANA10105,34 days,0,0
...,...,...,...,...,...
2511,DHA4,DHA469301,50 days,5,1
3990,PUMO,PUMO01302,10 days,5,0
2518,DHA1,DHA169101,8 days,5,2
7799,MANA,MANA12317,14 days,5,0


Looking at it this way, not really seeing anything substantial here when it comes to longer expeditions and more deaths. I think generally you'll see lower deaths with shorter expeditions, which is intuitive. But there's a lot of data here, and just sorting averages like this feels like it's missing context. 

Maybe if we group by peak (and expand our window) and look at things by rates we'll see something:

In [234]:
association_check = deaths_vs_length[['peak_id','length_of_expedition', 'member_deaths', 'hired_staff_deaths']
].groupby('peak_id').agg(
    'mean', numeric_only = False)
    
association_check.nlargest(20, 'hired_staff_deaths')


,length_of_expedition,member_deaths,hired_staff_deaths
peak_id,,,
JETH,36 days 00:00:00,0.000000,1.000000
DHA4,52 days 12:00:00,3.500000,0.500000
LEON,36 days 09:36:00,0.200000,0.400000
PK29,36 days 00:00:00,0.333333,0.333333
GANG,27 days 10:06:18.947368421,0.263158,0.315789
CHRW,27 days 18:00:00,0.000000,0.250000
DHA2,35 days 15:00:00,0.000000,0.250000
HIME,40 days 03:12:00,0.333333,0.200000
NAG1,18 days 14:24:00,0.000000,0.200000


In [235]:
association_check.nlargest(20, 'member_deaths')

,length_of_expedition,member_deaths,hired_staff_deaths
peak_id,,,
DHA4,52 days 12:00:00,3.500000,0.500000
HIMN,28 days 12:00:00,1.500000,0.000000
KJRN,39 days 00:00:00,1.000000,0.000000
GAN4,21 days 09:36:00,0.800000,0.000000
FIRN,23 days 12:00:00,0.500000,0.000000
GAN2,35 days 18:00:00,0.375000,0.000000
YALU,39 days 17:27:16.363636363,0.363636,0.000000
ANN3,30 days 00:00:00,0.333333,0.000000
HIME,40 days 03:12:00,0.333333,0.200000


In [236]:
association_check.nlargest(20, 'length_of_expedition')

,length_of_expedition,member_deaths,hired_staff_deaths
peak_id,,,
DHA5,67 days 12:00:00,0.000000,0.0
TSUR,62 days 12:00:00,0.000000,0.0
DZAN,60 days 00:00:00,0.000000,0.0
LHOM,60 days 00:00:00,0.000000,0.0
KHUM,54 days 00:00:00,0.000000,0.0
SNOW,54 days 00:00:00,0.000000,0.0
MANP,53 days 00:00:00,0.000000,0.0
DHA4,52 days 12:00:00,3.500000,0.5
CTSE,52 days 00:00:00,0.000000,0.0


Still not seeing anything in terms of expedition ~ death, aside from the obvious "it's more difficult to die the less time you spend on the mountain." 

We can also use those z-scores though as one more check. First, we have to add the z-scores back in.

In [302]:
zscores = dates['peak_length_score']
deaths_vs_length['zscores'] = zscores

#look at data for everest
deaths_vs_length[deaths_vs_length['peak_id'] == 'EVER'].nlargest(15, 'zscores')

,peak_id,expedition_id,length_of_expedition,member_deaths,hired_staff_deaths,zscores
468,EVER,EVER87401,133 days,0,0,6.713714
7495,EVER,EVER12128,107 days,0,0,4.818632
2623,EVER,EVER98101,104 days,0,0,4.599969
469,EVER,EVER88403,102 days,0,0,4.454193
2754,EVER,EVER21101,96 days,1,1,4.016867
1076,EVER,EVER91401,94 days,0,0,3.871091
1844,EVER,EVER81101,90 days,0,0,3.579540
1649,EVER,EVER95201,84 days,0,0,3.142214
459,EVER,EVER87301,83 days,1,0,3.069326
897,EVER,EVER87302,83 days,0,0,3.069326


Even within the distribution of Everest expeditions, longer ones don't appear to necessarily result in more deaths. 